<a href="https://colab.research.google.com/github/saji89lulu/automatizar_envio_email/blob/main/Automatizar_env%C3%ADo_de_Emails_leyendo_un_archivo_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Email Personalization Project**

This project aims to automate the generation of personalized email messages based on a list of email addresses provided in a CSV file. The emails are grouped by domain, and each user receives a customized message.



In [1]:
import re
import pandas as pd

In [11]:
# Clase Email para manejar los emails
class Email:

    def proveedor_usuarios(self, lista):
        self.lista = lista
        lista_dividida = []

        # Separo los emails por el carácter @
        for email in self.lista:
            result = re.split(r'@', email)  # Divido email en usuario y dominio
            lista_dividida.append(result)

        proveedor = {}  # Diccionario para organizar por el dominio
        # Itero en la lista para agrupar los usuarios por dominio
        for datos in lista_dividida:
            usuario, dominio = datos  # Desempaqueto usuario y dominio
            if dominio not in proveedor:
                proveedor[dominio] = []  # Si no está el dominio, lo añado como clave
            proveedor[dominio].append(usuario)  # Añado el usuario al dominio correspondiente

        return proveedor

    def mensaje(self, texto):
        self.texto = texto
        with open('mensaje_generico.txt', 'w') as f:
            f.write(self.texto)
        with open('mensaje_generico.txt', 'r') as f:
            return f.read()

    def preparar_envio(self, diccionario, proveedor, generico):
        self.diccionario = diccionario
        self.proveedor = proveedor
        self.generico = generico
        acumulado = 0  # Inicializamos acumulado

        # Verifico si el proveedor está en el diccionario
        if self.proveedor in self.diccionario:
            usuarios = self.diccionario[self.proveedor]  # Leer los valores del proveedor
            acumulado = len(usuarios)  # La cantidad de usuarios que coinciden con el proveedor
            # Itero en los valores (nombres) del diccionario
            for usuario in usuarios:
                # Reemplazo las variables en el mensaje personalizado
                usuario=usuario.split('.')[0] # Selecciono el primer nombre del email
                mensaje_personalizado = re.sub(r'{nombre}', usuario.capitalize(), self.generico)
                mensaje_personalizado = re.sub(r'{proveedor}', self.proveedor, mensaje_personalizado)
                # Escribo el mensaje personalizado en el archivo de salida
                with open(f'{usuario}@{proveedor}.txt', 'w') as archivo_salida:
                    archivo_salida.write(mensaje_personalizado)
                    archivo_salida.close()
        else:
            print('No se encontró proveedor')

        return f'Se han creado {acumulado} emails personalizados de cuentas de {self.proveedor}'

# Leer el archivo Excel o CSV con pandas para extraer la lista de emails
def cargar_emails_desde_csv(ruta_archivo):
    df = pd.read_csv(ruta_archivo)  # Lee el archivo CSV
    emails = df['Email'].tolist()  # Convierte la columna 'email' a lista
    return emails

# Ruta del archivo CSV
ruta_archivo_csv = 'emails.csv'

# Cargar los emails del archivo CSV
Lista_emails = cargar_emails_desde_csv(ruta_archivo_csv)

# Proceso principal
email_obj = Email()
dict_emails = email_obj.proveedor_usuarios(Lista_emails)
generico = email_obj.mensaje('Buenos días {nombre},\nGracias por elegir {proveedor} como tu proveedor de mensajes.\nUn cordial saludo')

# Preparar el envío de emails
# Iterar sobre todos los dominios y generar emails para todos
for dominio, usuarios in dict_emails.items():
    resultado = email_obj.preparar_envio(dict_emails, dominio, generico)
    print(resultado)


{'vmail.es': ['garcia.lopez'], 'gagoo.es': ['sanchez.mariano'], 'yotmail.com': ['sahara.quinones'], 'youtlook.com': ['noemi.cabrera']}
Los dominios son: vmail.es
Se han creado 1 emails personalizados de cuentas de vmail.es
Los dominios son: gagoo.es
Se han creado 1 emails personalizados de cuentas de gagoo.es
Los dominios son: yotmail.com
Se han creado 1 emails personalizados de cuentas de yotmail.com
Los dominios son: youtlook.com
Se han creado 1 emails personalizados de cuentas de youtlook.com
